In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import bs4.element
import datetime

In [2]:
# 네이버 금융 속보 url 리스트 만들기
# https://news.naver.com/main/list.naver?mode=LSD&mid=sec&sid1=101
# 100 : 정치, 101: 경제, 102 : 사회
url_list = []
for sid in ['100', '101', '102']:
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec" \
                + "&sid1=" \
                + sid \
             
    url_list.append(sec_url)

print(url_list)

['https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100', 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101', 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=102']


In [3]:
def ex_tag(page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
    
    ## 1.
    headers = {"User-Agent": "Mozilla/5.0" \
           "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
            "Chrome/114.0.0.0 Safari/537.36"}
    html = requests.get(page, headers=headers)
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")
    
    ## 2.
    tag_lst = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_lst.append(a["href"])
                
    return tag_lst

In [4]:
# 리스트 선언
news_titles = []
news_url =[]
news_contents =[]
news_dates = []

In [5]:
#뉴스 크롤러 실행 - url 받아오기
for i in url_list:
    news_url.append(ex_tag(i))

print(news_url)

[['https://n.news.naver.com/mnews/article/052/0001905776?sid=102', 'https://n.news.naver.com/mnews/article/055/0001070682?sid=102'], ['https://n.news.naver.com/mnews/article/052/0001905776?sid=102', 'https://n.news.naver.com/mnews/article/055/0001070682?sid=102'], ['https://n.news.naver.com/mnews/article/003/0011955566?sid=102', 'https://n.news.naver.com/mnews/article/003/0011955565?sid=102', 'https://n.news.naver.com/mnews/article/003/0011955565?sid=102', 'https://n.news.naver.com/mnews/article/087/0000981135?sid=102', 'https://n.news.naver.com/mnews/article/087/0000981135?sid=102', 'https://n.news.naver.com/mnews/article/087/0000981134?sid=102', 'https://n.news.naver.com/mnews/article/469/0000748110?sid=102', 'https://n.news.naver.com/mnews/article/469/0000748110?sid=102', 'https://n.news.naver.com/mnews/article/020/0003507668?sid=102', 'https://n.news.naver.com/mnews/article/020/0003507668?sid=102', 'https://n.news.naver.com/mnews/article/001/0014047283?sid=102', 'https://n.news.nav

In [6]:
#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

print(news_url)

  0%|          | 0/43 [00:00<?, ?it/s]

[['https://n.news.naver.com/mnews/article/052/0001905776?sid=102', 'https://n.news.naver.com/mnews/article/055/0001070682?sid=102'], ['https://n.news.naver.com/mnews/article/052/0001905776?sid=102', 'https://n.news.naver.com/mnews/article/055/0001070682?sid=102'], ['https://n.news.naver.com/mnews/article/003/0011955566?sid=102', 'https://n.news.naver.com/mnews/article/003/0011955565?sid=102', 'https://n.news.naver.com/mnews/article/003/0011955565?sid=102', 'https://n.news.naver.com/mnews/article/087/0000981135?sid=102', 'https://n.news.naver.com/mnews/article/087/0000981135?sid=102', 'https://n.news.naver.com/mnews/article/087/0000981134?sid=102', 'https://n.news.naver.com/mnews/article/469/0000748110?sid=102', 'https://n.news.naver.com/mnews/article/469/0000748110?sid=102', 'https://n.news.naver.com/mnews/article/020/0003507668?sid=102', 'https://n.news.naver.com/mnews/article/020/0003507668?sid=102', 'https://n.news.naver.com/mnews/article/001/0014047283?sid=102', 'https://n.news.nav

In [7]:
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

  0%|          | 0/43 [00:00<?, ?it/s]

NameError: name 're' is not defined

In [ ]:
###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H%M%S')),encoding='utf-8-sig',index=False)

In [ ]:
for sid in ['100']:#, '101', '102']:
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec" \
                + "&sid1=" \
                + sid
    print("section url : ", sec_url)
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기
    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        # title : 뉴스 제목, news_url : 뉴스 URL, image_url : 이미지 URL
        news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") , 
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href')
        }
        print(news_info)